## GLHF Tutorial

Before you start, please read the README.md file first.

Despite the lack of extensive documentation in this project, it is well-typed. This makes static type checkers and hover information boxes especially helpful when using editors or IDEs like VSCode or PyCharm. Take advantage of these tools to understand and navigate the codebase more efficiently.

## Minimal Example of a SurrenderBot

In [2]:
from secrets import token_urlsafe

from glhf.base import Bot, ClientProtocol


class SurrenderBot(Bot):
    """
    A bot that surrenders the game after a certain number of turns.
    """

    async def run(self, client: ClientProtocol) -> None:
        # Join the default room or a random room if no default room is specified.
        client.join_private(self.default_room or token_urlsafe(3))

        # Listen for queue updates and send a force start request.
        async for data in self.queue_update:
            if not data["isForcing"]:
                client.set_force_start(True)

        # Listen for game updates and surrender after 50 turns.
        async for data in self.game_update:
            if data["turn"] == 1:
                client.surrender()

        # Get the game result
        if self.game_won.get():
            print("win")
        elif self.game_lost.get():
            print("lose")
        else:
            print("never")

## How to Run Your Bot?

### With Programming Style
- Messy imports, but clear type hints

In [3]:
import asyncio
from glhf.gui import PygameGUI
from glhf.server import SocketioServer
from glhf.app import start


def main(is_jupyter: bool = True) -> None:
    USERID = "123"
    USERNAME = "[Bot]123"

    server = SocketioServer()
    bot = SurrenderBot(USERID, USERNAME)
    gui = PygameGUI()

    gui.register(bot)

    # Use the low-level API due to limitations of Jupyter Notebook.

    if is_jupyter:
        coro = start(server, [bot], gui)
        loop = asyncio.get_event_loop()

        if loop.is_running():
            task = loop.create_task(coro)
            task.add_done_callback(lambda t: t.result())
        else:
            loop.run_until_complete(coro)
        return

    # Simply use the high-level API in regular scripts.

    asyncio.run(start(server, [bot], gui))


if __name__ == "__main__":
    main()


https://generals.io/games/BKjx
lose


### With the `APP` class

- Minimal imports, but potential issues
- It's primarily designed for CLI interactions

In [11]:
from glhf.app import APP

# You should define your bot class in the same file,
# or import it from another file.


def main() -> None:
    # Replace USERID and USERNAME with yours.
    USERID = "123"
    USERNAME = "[Bot]123"

    # Instantiate the APP class to manage workflow.
    app = APP()

    # Set server to "socketio" for full support.
    app.server("socketio")

    # Alternatively, choose "local" server for limited simulation:
    # - 1v1 game starts immediately when set force start
    # - no fog of war
    # - game stops after 50 turns

    # Add the "SurrenderBot" to the bot list.
    app.bot_add("SurrenderBot", userid=USERID, username=USERNAME)

    # Bind GUI to the first one in the bot list.
    # Comment out to disable GUI.
    app.gui(0)

    # Start the game.
    # URL printed if using socketio server.
    app.start()


if __name__ == "__main__":
    main()


+ SurrenderBot(id='123', name='[Bot]123', default_room='')

https://generals.io/games/x9x2
lose


### With `command` function

- Minimal lines of code.
- It's a CLI built upon the `APP` class.
- Depends on [python-fire](https://github.com/google/python-fire), which is simple to use, but the user experience could be better. Plans are in place to improve this.

In [ ]:
from glhf.app import command

# You should define your bot class in the same file,
# or import it from another file


if __name__ == "__main__":
    # try input "-- --help" to see all available commands
    # input ctrl+z to exit
    # not working in Jupyter Notebook
    command()


## Protocols

The output below shows all the methods implemented within the bot and client, which are the key components you'll interact with.

If your class structure aligns with the correct protocols, everything should work as expected.

These methods are simply interfaces for the official game APIs. If you're not planning to create your own server and client, you can use the ones provided.

To learn what a Protocol is, check out [this page](https://docs.python.org/3/library/typing.html#typing.Protocol).

For more information on these methods, check out [this page](https://web.archive.org/web/20230112021323/https://dev.generals.io/api).

In [10]:
from rich import inspect
from glhf.base import BotProtocol, ClientProtocol

inspect(BotProtocol, methods=True)
inspect(ClientProtocol, methods=True)

╭────────────────────────────── <class 'glhf.base.BotProtocol'> ──────────────────────────────╮
│ class BotProtocol(*args, **kwargs):                                                         │
│                                                                                             │
│ BotProtocol(*args, **kwargs)                                                                │
│                                                                                             │
│            gui = None                                                                       │
│   chat_message = def chat_message(self, chat_room: 'str', data: 'dict[str, Any]') -> 'Any': │
│      game_lost = def game_lost(self) -> 'Any':                                              │
│      game_over = def game_over(self) -> 'Any':                                              │
│     game_start = def game_start(self, data: 'GameStartDict') -> 'Any':                      │
│    game_update = def game_update(self, data: 'GameUpdateDict') -> 'Any':                    │
│       game_won = def game_won(self) -> 'Any':                                               │
│         notify = def notify(self, data: 'Any') -> 'Any':                                    │
│ pre_game_start = def pre_game_start(self) -> 'Any':                                         │
│   queue_update = def queue_update(self, data: 'QueueUpdateDict') -> 'Any':                  │
│           rank = def rank(self, data: 'dict[str, int]') -> 'Any':                           │
│            run = async def run(self, client: 'ClientProtocol') -> 'None':                   │
│          stars = def stars(self, data: 'dict[str, float]') -> 'Any':                        │
│          start = async def start(self, server: 'ServerProtocol') -> 'None':                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── <class 'glhf.base.ClientProtocol'> ─────────────────────────────────╮
│ class ClientProtocol(*args, **kwargs):                                                              │
│                                                                                                     │
│ Base class for protocol classes.                                                                    │
│                                                                                                     │
│          attack = def attack(self, start: 'int', end: 'int', is50: 'bool') -> 'asyncio.Task[None]': │
│    join_private = def join_private(self, queue_id: 'str') -> 'asyncio.Task[None]':                  │
│      leave_game = def leave_game(self) -> 'asyncio.Task[None]':                                     │
│ set_force_start = def set_force_start(self, do_force: 'bool') -> 'asyncio.Task[None]':              │
│    set_username = def set_username(self) -> 'asyncio.Task[None]':                                   │
│  stars_and_rank = def stars_and_rank(self) -> 'asyncio.Task[None]':                                 │
│       surrender = def surrender(self) -> 'asyncio.Task[None]':                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯

## WIP